### Cleaning DUV data


https://statistik.d-u-v.org/getresulteventalltime.php?event=25042

DUV data contains each runners Year of Birth (YOB)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import re

In [2]:
# Function to remove special characters
def normalize_name(name):
    # Normalize to remove accents and special characters
    name = unicodedata.normalize('NFD', name)
    # Encode to ASCII and ignore errors, then decode back to string
    name = name.encode('ascii', 'ignore').decode('utf-8')
    return name

In [3]:
TORX_duv_dict = {}

races = [ #'TOR130',
         'TOR330',
#          'TOR450'
        ]

for race in races:
    file = f'{race} Data/2. DUV/{race}_multiple.xlsx'
    df = pd.read_excel(file)

    df = df[['Name', 'Nat.', 'M/F', 'YOB', 'Date','Performance' ]]

    # renaming columns so it is easier later to compare
    df = df.rename(columns={"Nat.": "DUV_ationality",
                            "M/F": "Sex",
                            }) 
    

    ##########################################
    
    # When read in, YOB is a number, needs to be changed to a string
    df['YOB'] = df['YOB'].astype('str')    

    ##########################################

    for column in df.columns:
        # Clean up the 'Name' column by stripping leading/trailing spaces and non-breaking spaces
        df[column] = df[column].astype('str')
        df[column] = df[column].str.strip()
        df[column] = df[column].str.replace(',', '')
        df[column] = df[column].str.replace('\'', ' ')
        df[column] = df[column].str.replace(r"\s+", " ")

    df['Name'] = df['Name'].str.replace('-', ' ')
    # dropping all irregular letters like in Spanish
    df['Name'] = df['Name'].apply(normalize_name)
    # Title Case
    df['Name'] = df['Name'].str.title()

    ##########################################
    print(df['Date'].unique())
    # Assuming Date format like "08.-14.09.2019"
    df[['Start Day', 'End Day', 'Month', 'Year']] = df['Date'].str.extract(r'(\d{2})\.-(\d{2})\.(\d{2})\.(\d{4})')

    if race == 'TOR450':
        df['Start Date'] = df['Year'] + '.' + df['Month'] + '.' + df['Start Day'] + ' 20:00:00'

    elif race== 'TOR330':
        # sweeping assumption that TOR330 starts at 10am
        df['Start Date'] = df['Year'] + '.' + df['Month'] + '.' + df['Start Day'] + ' 10:00:00'
    # Combine them into desired Start Date and End Date format if needed
    elif race== 'TOR130':
        # sweeping assumption that TOR330 starts at 10am
        df['Start Date'] = df['Year'] + '.' + df['Month'] + '.' + df['Start Day'] + ' 21:00:00'
    # Combine them into desired Start Date and End Date format if needed
#     print(df['Start Date'].unique())

    df['Start Date'] = pd.to_datetime(df['Start Date'])

    ##########################################
    # Race 
    df['Race'] = f'{race}'

    ##########################################
    # changing the performance to a timedelta
    df['Performance'] = pd.to_timedelta(df['Performance'], errors='coerce')
    #finding out the time that participants finished so the dataset could be used in Tableau
    df['Finish Time for Tableau'] = df['Start Date'] + df['Performance']  
#     print(df.head())

    ##########################################
    # Create 'DOB' column by adding '01/01/' prefix to 'YOB'
    df['DOB'] = '01/01/' + df['YOB'].astype(str)

    # Replace '01/01/' without a following year with NaN
    df['DOB'] = df['DOB'].replace(r'^01/01/$', np.nan, regex=True)

    # Convert to datetime, with errors='coerce' to handle NaT cases
    df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')


    # Drop the intermediate columns if they're no longer needed
    df = df.drop(columns=['Start Day', 'End Day', 'Month', 'Date', 'YOB'])


    TORX_duv_dict[f'{race}_duv'] = df
    print(f'{race} DUV data was read in ')

C:\Users\Karina\AppData\Local\Temp\ipykernel_5328\3964185681.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace(r"\s+", " ")


['10.-16.09.2023' '12.-18.09.2021' '10.-17.09.2017' '08.-14.09.2024'
 '08.-15.09.2013' '11.-17.09.2022' '07.-14.09.2014' '08.-14.09.2019'
 '09.-16.09.2018' '11.-18.09.2016' '09.-16.09.2012' '11.-17.09.2011'
 '13.-20.09.2015' '12.-18.09.2010']
TOR330 DUV data was read in 


In [4]:
# Append using pd.concat
TORX_duv_df = pd.concat(TORX_duv_dict)

TORX_duv_df = TORX_duv_df[[
    'Name', 'DUV_ationality', 'Sex',  'DOB', 
    'Start Date', 'Performance', 
    'Race', 'Year',
#         'Race_Year', 'Performance_Race'
]].reset_index(drop = True)


TORX_duv_df[TORX_duv_df['Name'].str.contains('D Haene Francois')]

,Name,DUV_ationality,Sex,DOB,Start Date,Performance,Race,Year
4,D Haene Francois,FRA,M,1985-01-01,2024-09-08 10:00:00,2 days 21:08:32,TOR330,2024


In [5]:
TORX_duv_df[[
    'Name', 'DUV_ationality', 'Sex',  'DOB', 
    'Start Date', 'Performance', 
    'Race', 'Year',
#         'Race_Year', 'Performance_Race'
]][TORX_duv_df['Name'].str.contains('Jonina')]

,Name,DUV_ationality,Sex,DOB,Start Date,Performance,Race,Year
4200,Jonina Karina,IRL,F,1992-01-01,2023-09-10 10:00:00,6 days 00:46:43,TOR330,2023


### Extracting Start Date so can be used for other Datasets
But needs to be dropped in this one because it is impossible to detect the start time for each individual. No bib numbers makes it impossible to detect which wave the individuals was in.  

In [6]:

TORX_date_info_df = TORX_duv_df[[
    'Race', 'Year',
    'Start Date',
    
]].copy()

# Dropping all redundant rows
TORX_date_info_df.drop_duplicates( keep='first', inplace=True, ignore_index=False)


# Extract the day of the week
TORX_date_info_df['Day_of_Week Start_Date'] = TORX_date_info_df['Start Date'].dt.day_name()

TORX_date_info_df = TORX_date_info_df.reset_index(drop= True)

print(TORX_date_info_df[['Race', 'Year', 'Start Date', 'Day_of_Week Start_Date']])

TORX_date_info_df.to_excel(f'TORX DUV dates/TORX_duv_dates_df.xlsx' , index = False)


      Race  Year          Start Date Day_of_Week Start_Date
0   TOR330  2023 2023-09-10 10:00:00                 Sunday
1   TOR330  2021 2021-09-12 10:00:00                 Sunday
2   TOR330  2017 2017-09-10 10:00:00                 Sunday
3   TOR330  2024 2024-09-08 10:00:00                 Sunday
4   TOR330  2013 2013-09-08 10:00:00                 Sunday
5   TOR330  2022 2022-09-11 10:00:00                 Sunday
6   TOR330  2014 2014-09-07 10:00:00                 Sunday
7   TOR330  2019 2019-09-08 10:00:00                 Sunday
8   TOR330  2018 2018-09-09 10:00:00                 Sunday
9   TOR330  2016 2016-09-11 10:00:00                 Sunday
10  TOR330  2012 2012-09-09 10:00:00                 Sunday
11  TOR330  2011 2011-09-11 10:00:00                 Sunday
12  TOR330  2015 2015-09-13 10:00:00                 Sunday
13  TOR330  2010 2010-09-12 10:00:00                 Sunday


### This data might be used for the data visualisations beyond than 2021

In [7]:
# Calculate total seconds and create 'Performance_Seconds' column
TORX_duv_df['Performance_Seconds'] = TORX_duv_df['Performance'].dt.total_seconds()

TORX_duv_df[['Name', 'Performance', 'Performance_Seconds']]

,Name,Performance,Performance_Seconds
0,Colle Franco Leo,2 days 18:39:16,239956.0
1,Colle Franco Leo,2 days 18:43:57,240237.0
2,Dominguez Ledo Javier,2 days 19:52:15,244335.0
3,Russi Jonas,2 days 21:03:00,248580.0
4,D Haene Francois,2 days 21:08:32,248912.0
...,...,...,...
5379,Landi Riccardo,6 days 07:37:19,545839.0
5380,Mondin Placido,6 days 08:05:50,547550.0
5381,Forestiero Alessandro,6 days 08:14:00,548040.0
5382,Tibaldi Erik,6 days 08:15:44,548144.0


In [8]:
save_TORX_duv_df = TORX_duv_df.copy()

save_TORX_duv_df = save_TORX_duv_df[[
    'Name', 'DUV_ationality', 'Sex',  'DOB', 'Performance','Performance_Seconds', 
    'Race', 'Year']]

# # Save the result
save_TORX_duv_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_duv.xlsx' , index = False)

## EDA of TOR440 and TOR 330

In [9]:
TORX_duv_df[TORX_duv_df['Name'].str.contains('Colle Franco')]

,Name,DUV_ationality,Sex,DOB,Start Date,Performance,Race,Year,Performance_Seconds
0,Colle Franco Leo,ITA,M,1978-01-01,2023-09-10 10:00:00,2 days 18:39:16,TOR330,2023,239956.0
1,Colle Franco Leo,ITA,M,1978-01-01,2021-09-12 10:00:00,2 days 18:43:57,TOR330,2021,240237.0
11,Colle Franco Leo,ITA,M,1978-01-01,2014-09-07 10:00:00,2 days 23:49:10,TOR330,2014,258550.0
13,Colle Franco Leo,ITA,M,1978-01-01,2013-09-08 10:00:00,3 days 00:05:23,TOR330,2013,259523.0
18,Colle Franco Leo,ITA,M,1978-01-01,2018-09-09 10:00:00,3 days 02:03:00,TOR330,2018,266580.0
85,Colle Franco Leo,ITA,M,1978-01-01,2012-09-09 10:00:00,3 days 12:15:59,TOR330,2012,303359.0


In [10]:
# Combine 'Performance' and 'Race' into a single column
TORX_duv_df['Performance_Race'] = TORX_duv_df['Performance'].astype('str') + " " + TORX_duv_df['Race']

# Pivot the DataFrame
TORX_duv_pivot_df = TORX_duv_df.pivot(index='Name', columns='Year', values='Performance_Race')

# Fill missing values with "-"
TORX_duv_pivot_df = TORX_duv_pivot_df.fillna('-')

# Reset the index to make 'Name' a column again
TORX_duv_pivot_df = TORX_duv_pivot_df.reset_index()

# Save the result
# TORX_duv_pivot_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/TORX_duv_pivot_df.xlsx' , index = False)

TORX_duv_pivot_df

Year,Name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022,2023,2024
0,Aakre Eivind,-,-,-,-,-,-,-,-,6 days 00:45:33 TOR330,-,-,-,-,-
1,Aarons Jamie,-,-,-,-,-,-,5 days 11:26:21 TOR330,-,3 days 23:54:35 TOR330,-,-,-,-,-
2,Abanera Luca,-,-,-,-,-,-,6 days 01:48:21 TOR330,-,5 days 07:52:29 TOR330,-,-,-,-,-
3,Abate Francesco,-,-,-,5 days 05:52:57 TOR330,-,-,5 days 05:03:29 TOR330,-,-,-,-,-,-,-
4,Abbate Mauro,-,-,-,-,-,-,-,-,-,4 days 21:40:07 TOR330,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,Zucconi Francesco,-,-,4 days 13:39:31 TOR330,-,-,-,-,-,-,-,-,-,-,-
4175,Zumeta Artola Garikoitz,-,-,-,-,-,-,5 days 04:20:05 TOR330,-,-,-,-,-,-,-
4176,Zuncheddu Matteo,-,-,-,-,-,-,-,-,5 days 04:11:59 TOR330,-,-,-,-,-
4177,Zurita Gimeno Nacho,-,-,-,-,-,-,-,-,-,-,-,-,-,4 days 12:51:28 TOR330
